In [114]:
!pip install yfinance
!pip install svm

In [115]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
import csv

In [116]:
from google.colab import files
uploaded = files.upload()

In [117]:
aapl = pd.read_csv("Quarterly_Data_AAPL.csv")
aapl.insert(1, "Ticker", "AAPL")
aapl

,Year,Ticker,Change_Net_Income,Change_Cash_Operations,Change_Cash_Investments,Change_Cash_Financials,Change_Cash_Equivalents
0,2014-12-27,AAPL,18024,33722,-21165,6923,5634
1,2015-03-28,AAPL,13569,19081,-22331,-15585,-4989
2,2015-06-25,AAPL,10677,14988,-11403,-2755,830
3,2015-09-26,AAPL,11124,13475,-1375,-6299,5801
4,2015-12-26,AAPL,18361,27463,-20450,-11444,-4431
5,2016-03-26,AAPL,10516,11601,-13660,6884,4825
6,2016-06-25,AAPL,7796,10634,-4470,-9441,-3277
7,2016-09-24,AAPL,9014,16126,-7397,-6482,2247
8,2016-12-31,AAPL,17891,27056,-19122,-12047,-4113
9,2017-04-01,AAPL,11029,12523,-14202,465,-1214


In [118]:
tsla = pd.read_csv("Quarterly_Data_TSLA.csv")
tsla.insert(1, "Ticker", "TSLA")
tsla

,Year,Ticker,Change_Net_Income,Change_Cash_Operations,Change_Cash_Investments,Change_Cash_Financials,Change_Cash_Equivalents
0,2015-03-31,TSLA,-154.181,-131.794,-432.344,186.156,-395.637
1,2015-06-30,TSLA,-184.227,-159.516,-422.837,218.351,-359.403
2,2015-09-30,TSLA,-229.858,-203.340,-404.090,893.978,275.363
3,2015-12-31,TSLA,-320.397,-29.849,-414.280,225.038,-229.128
4,2016-03-31,TSLA,-282.267,-249.605,-233.819,715.435,244.881
5,2016-06-30,TSLA,-293.188,150.336,-319.854,1976.584,1804.512
6,2016-09-30,TSLA,21.878,423.649,-268.006,-320.870,-162.044
7,2016-12-31,TSLA,-219.469,-448.209,-594.751,1372.827,308.959
8,2017-03-31,TSLA,-397.181,-69.811,-926.943,1598.749,613.377
9,2017-06-30,TSLA,-401.427,-200.172,-1215.216,428.767,-970.669


In [119]:
#Retrieve adjusted close for each date we have
def get_adj_close(df, tckr):
    dates = list(df["Year"])
    close = np.array([])
    start = ""

    for i in range(len(dates) - 1):
        start = dates[i]
        end = dates[i + 1]
        adj = yf.download(tckr, start = start, end = end)["Adj Close"]
        close = np.append(close, adj[0])

        if i is len(dates) - 2:
            close = np.append(close, adj[-1])
    return close

In [120]:
aapl["Adj_Close"] = get_adj_close(aapl, "AAPL")
tsla["Adj_Close"] = get_adj_close(tsla, "TSLA")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [121]:
aapl["%Change"] = aapl.Adj_Close.pct_change()
aapl
# aapl["%Change"] = np.where(aapl.Adj_Close.pct_change() > 0, "BULLISH", "BEARISH")
# aapl

,Year,Ticker,Change_Net_Income,Change_Cash_Operations,Change_Cash_Investments,Change_Cash_Financials,Change_Cash_Equivalents,Adj_Close,%Change
0,2014-12-27,AAPL,18024,33722,-21165,6923,5634,25.902882,NaN
1,2015-03-28,AAPL,13569,19081,-22331,-15585,-4989,28.849669,0.113763
2,2015-06-25,AAPL,10677,14988,-11403,-2755,830,29.229227,0.013156
3,2015-09-26,AAPL,11124,13475,-1375,-6299,5801,25.893421,-0.114126
4,2015-12-26,AAPL,18361,27463,-20450,-11444,-4431,24.704502,-0.045916
5,2016-03-26,AAPL,10516,11601,-13660,6884,4825,24.459534,-0.009916
6,2016-06-25,AAPL,7796,10634,-4470,-9441,-3277,21.532110,-0.119684
7,2016-09-24,AAPL,9014,16126,-7397,-6482,2247,26.550533,0.233067
8,2016-12-31,AAPL,17891,27056,-19122,-12047,-4113,27.459938,0.034252
9,2017-04-01,AAPL,11029,12523,-14202,465,-1214,34.120541,0.242557


In [122]:
tsla["%Change"] = tsla.Adj_Close.pct_change()

# tsla["%Change"] = np.where(tsla.Adj_Close.pct_change() > 0, "BULLISH", "BEARISH")
tsla

,Year,Ticker,Change_Net_Income,Change_Cash_Operations,Change_Cash_Investments,Change_Cash_Financials,Change_Cash_Equivalents,Adj_Close,%Change
0,2015-03-31,TSLA,-154.181,-131.794,-432.344,186.156,-395.637,37.754002,NaN
1,2015-06-30,TSLA,-184.227,-159.516,-422.837,218.351,-359.403,53.652000,0.421094
2,2015-09-30,TSLA,-229.858,-203.340,-404.090,893.978,275.363,49.680000,-0.074033
3,2015-12-31,TSLA,-320.397,-29.849,-414.280,225.038,-229.128,48.001999,-0.033776
4,2016-03-31,TSLA,-282.267,-249.605,-233.819,715.435,244.881,45.953999,-0.042665
5,2016-06-30,TSLA,-293.188,150.336,-319.854,1976.584,1804.512,42.456001,-0.076120
6,2016-09-30,TSLA,21.878,423.649,-268.006,-320.870,-162.044,40.806000,-0.038864
7,2016-12-31,TSLA,-219.469,-448.209,-594.751,1372.827,308.959,43.397999,0.063520
8,2017-03-31,TSLA,-397.181,-69.811,-926.943,1598.749,613.377,55.660000,0.282548
9,2017-06-30,TSLA,-401.427,-200.172,-1215.216,428.767,-970.669,72.321999,0.299353


In [123]:
# #Combine dfs without NaN row
df = pd.concat([aapl[1:], tsla[1:]])
# df["%Change"] = np.where(df["%Change"] == "BULLISH", 1.0, 0.0)
df

,Year,Ticker,Change_Net_Income,Change_Cash_Operations,Change_Cash_Investments,Change_Cash_Financials,Change_Cash_Equivalents,Adj_Close,%Change
1,2015-03-28,AAPL,13569.000,19081.000,-22331.000,-15585.000,-4989.000,28.849669,0.113763
2,2015-06-25,AAPL,10677.000,14988.000,-11403.000,-2755.000,830.000,29.229227,0.013156
3,2015-09-26,AAPL,11124.000,13475.000,-1375.000,-6299.000,5801.000,25.893421,-0.114126
4,2015-12-26,AAPL,18361.000,27463.000,-20450.000,-11444.000,-4431.000,24.704502,-0.045916
5,2016-03-26,AAPL,10516.000,11601.000,-13660.000,6884.000,4825.000,24.459534,-0.009916
6,2016-06-25,AAPL,7796.000,10634.000,-4470.000,-9441.000,-3277.000,21.532110,-0.119684
7,2016-09-24,AAPL,9014.000,16126.000,-7397.000,-6482.000,2247.000,26.550533,0.233067
8,2016-12-31,AAPL,17891.000,27056.000,-19122.000,-12047.000,-4113.000,27.459938,0.034252
9,2017-04-01,AAPL,11029.000,12523.000,-14202.000,465.000,-1214.000,34.120541,0.242557
10,2017-07-01,AAPL,8717.000,8363.000,-3180.000,-1769.000,3414.000,34.213696,0.002730


In [124]:
#Split data (Train on 2020)
x_train = df[~df["Year"].str.contains("2020")].iloc[:,2:-1].values
y_train = df[~df["Year"].str.contains("2020")].iloc[:,-1:].values

x_test = df[df["Year"].str.contains("2020")].iloc[:,2:-1].values
y_test = df[df["Year"].str.contains("2020")].iloc[:,-1:].values

# x = df.iloc[:,1:-1].values
# y = df.iloc[:, -1].values
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

print(x_train)
print(y_train)
print(x_test)
print(y_test)

[[ 1.35690000e+04  1.90810000e+04 -2.23310000e+04 -1.55850000e+04
  -4.98900000e+03  2.88496685e+01]
 [ 1.06770000e+04  1.49880000e+04 -1.14030000e+04 -2.75500000e+03
   8.30000000e+02  2.92292271e+01]
 [ 1.11240000e+04  1.34750000e+04 -1.37500000e+03 -6.29900000e+03
   5.80100000e+03  2.58934212e+01]
 [ 1.83610000e+04  2.74630000e+04 -2.04500000e+04 -1.14440000e+04
  -4.43100000e+03  2.47045021e+01]
 [ 1.05160000e+04  1.16010000e+04 -1.36600000e+04  6.88400000e+03
   4.82500000e+03  2.44595337e+01]
 [ 7.79600000e+03  1.06340000e+04 -4.47000000e+03 -9.44100000e+03
  -3.27700000e+03  2.15321102e+01]
 [ 9.01400000e+03  1.61260000e+04 -7.39700000e+03 -6.48200000e+03
   2.24700000e+03  2.65505333e+01]
 [ 1.78910000e+04  2.70560000e+04 -1.91220000e+04 -1.20470000e+04
  -4.11300000e+03  2.74599380e+01]
 [ 1.10290000e+04  1.25230000e+04 -1.42020000e+04  4.65000000e+02
  -1.21400000e+03  3.41205406e+01]
 [ 8.71700000e+03  8.36300000e+03 -3.18000000e+03 -1.76900000e+03
   3.41400000e+03  3.4213

In [125]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(x_train, y_train)
y_predicted_lr = regressor.predict(x_test)

# round predictions
y_predicted_lr = np.where(y_test < 0.5, 0, 1).reshape(-1,1)
y_predicted_lr

array([[0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1]])

In [126]:
# y_train.shape[1]

X_orig = df.iloc[:,2:-1].values
y_orig = df.iloc[:,-1:].values
y_orig

array([[ 0.1137629 ],
       [ 0.01315643],
       [-0.1141257 ],
       [-0.04591587],
       [-0.00991594],
       [-0.11968435],
       [ 0.23306694],
       [ 0.03425185],
       [ 0.24255709],
       [ 0.00273017],
       [ 0.07605569],
       [ 0.12397913],
       [-0.02844754],
       [ 0.1273206 ],
       [ 0.21838324],
       [-0.30348326],
       [ 0.21757452],
       [ 0.05796968],
       [ 0.11546015],
       [ 0.30551055],
       [-0.12385173],
       [ 0.42364649],
       [ 0.24365559],
       [ 0.4210944 ],
       [-0.07403266],
       [-0.0337762 ],
       [-0.0426649 ],
       [-0.07611954],
       [-0.0388638 ],
       [ 0.06352005],
       [ 0.28254761],
       [ 0.29935319],
       [-0.05552942],
       [-0.06148797],
       [-0.21230468],
       [ 0.32711506],
       [-0.07273108],
       [ 0.07112968],
       [-0.1310697 ],
       [-0.21443393],
       [ 0.0603073 ],
       [ 0.73674598],
       [ 0.25259965],
       [ 1.0607061 ],
       [ 0.98650688],
       [ 0

In [144]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn import datasets, svm
from sklearn import preprocessing
from sklearn import utils

lab_enc = preprocessing.LabelEncoder()
encoded_tr = lab_enc.fit_transform(y_train.ravel())
encoded_ts = lab_enc.fit_transform(y_test.ravel())

# encoded

# print(utils.multiclass.type_of_target(y_orig.ravel()))

# print(utils.multiclass.type_of_target(y_orig.ravel().astype('int')))

# print(utils.multiclass.type_of_target(encoded))
svc = svm.SVC(kernel='linear')
k_fold = KFold(n_splits=5)

# [svc.fit(x_train[x_tr], encoded_tr[y_tr]).score(x_test[x_ts], encoded_ts[y_ts]) for x_tr, x_ts, y_tr, y_ts in zip(x_train, encoded_tr, x_test, encoded_ts)]

print(y_train.shape[0])
print(encoded.shape[0])
# y_train = encoded

# y_train

# accuracies = cross_val_score(estimator = regressor, X = x_train, y = encoded, cv = 3)
# print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
# print("Standard Deviation: {:.2f} %".format(accuracies.std()*100)+"\n\n")


IndexError: ignored

In [128]:
from sklearn.model_selection import cross_val_score
from sklearn import svm
import sklearn

print(sorted(sklearn.metrics.SCORERS.keys()))


['accuracy', 'adjusted_mutual_info_score', 'adjusted_rand_score', 'average_precision', 'balanced_accuracy', 'completeness_score', 'explained_variance', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'fowlkes_mallows_score', 'homogeneity_score', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted', 'max_error', 'mutual_info_score', 'neg_brier_score', 'neg_log_loss', 'neg_mean_absolute_error', 'neg_mean_gamma_deviance', 'neg_mean_poisson_deviance', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_median_absolute_error', 'neg_root_mean_squared_error', 'normalized_mutual_info_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'r2', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'roc_auc', 'roc_auc_ovo', 'roc_auc_ovo_weighted', 'roc_auc_ovr', 'roc_auc_ovr_weighted', 'v_measure_score']


In [145]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = regressor, X = x_train, y = encoded_tr, cv = 3)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100)+"\n\n")


from sklearn.model_selection import GridSearchCV
# parameters = [{'fit_intercept':['True', 'False'], 'normalize':['True', 'False'], 'positive':['True', 'False']}]
parameters = [{'fit_intercept':['True', 'False'], 'normalize':['True', 'False']}]
grid_search = GridSearchCV(estimator = regressor, param_grid = parameters, scoring = 'neg_root_mean_squared_error', cv = 5, n_jobs = -1)

grid_search.fit(x_train, y_train.ravel())
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
print("Best Accuracy: {:.2f} %".format(best_accuracy*100))
print("Best Parameters:", best_parameters)

Accuracy: -177810.37 %
Standard Deviation: 251446.62 %


Best Accuracy: -21.30 %
Best Parameters: {'fit_intercept': 'True', 'normalize': 'True'}


In [130]:
# from sklearn.model_selection import cross_val_score
# accuracies = cross_val_score(estimator = regressor, X = x_train, y = y_train, cv = 3)
# print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
# print("Standard Deviation: {:.2f} %".format(accuracies.std()*100)+"\n\n")


# from sklearn.model_selection import GridSearchCV
# # parameters = [{'fit_intercept':['True', 'False'], 'normalize':['True', 'False'], 'positive':['True', 'False']}]
# parameters = [{'fit_intercept':['True', 'False'], 'normalize':['True', 'False']}]
# grid_search = GridSearchCV(estimator = regressor, param_grid = parameters, scoring = 'neg_root_mean_squared_error', cv = 5, n_jobs = -1)

# grid_search.fit(x_train, y_train.ravel())
# best_accuracy = grid_search.best_score_
# best_parameters = grid_search.best_params_
# print("Best Accuracy: {:.2f} %".format(best_accuracy*100))
# print("Best Parameters:", best_parameters)

In [131]:
# from sklearn.model_selection import cross_val_score
# from sklearn import svm
# import sklearn
# # svc = svm.SVC(C=1, kernel='linear')
# clf = svm.SVC(random_state=0)
# accuracies = cross_val_score(estimator = clf, X = x_train, y = y_train.ravel(), cv = 10)
# print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
# print("Standard Deviation: {:.2f} %".format(accuracies.std()*100)+"\n\n")


# from sklearn.model_selection import GridSearchCV
# parameters = [{'fit_intercept':['True', 'False'], 'normalize':['True', 'False'], 'positive':['True', 'False']}]
# grid_search = GridSearchCV(estimator = regressor, param_grid = parameters, scoring = 'max_error', cv = 5, n_jobs = -1)
# print(sorted(sklearn.metrics.SCORERS.keys()))

# grid_search.fit(x_train, y_train)
# best_accuracy = grid_search.best_score_
# best_parameters = grid_search.best_params_
# print("Best Accuracy: {:.2f} %".format(best_accuracy*100))
# print("Best Parameters:", best_parameters)

In [132]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression(fit_intercept=True, normalize=True)
regressor.fit(x_train, y_train)
y_predicted_lr = regressor.predict(x_test)

# round predictions
y_predicted_lr = np.where(y_predicted < 0.5, 0, 1).reshape(-1,1)
y_predicted_lr

TypeError: ignored

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly_reg = PolynomialFeatures(degree = 4)
X_poly = poly_reg.fit_transform(x_train)
lin_reg_2 = LinearRegression()
lin_reg_2.fit(X_poly, y_train)

In [ ]:
y_predicted = lin_reg_2.predict(X_poly)

# round predictions
y_predicted = np.array([[round(x[0]) for x in y_predicted]]).reshape(-1,1)
y_predicted

In [ ]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = lin_reg_2, X = x_train, y = y_train.ravel(), cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100)+"\n\n")


from sklearn.model_selection import GridSearchCV
parameters = [{'fit_intercept':[True, False], 'normalize':[True, False], 'positive':[True, False]}]
grid_search = GridSearchCV(estimator = lin_reg_2, param_grid = parameters, scoring = 'neg_root_mean_squared_error', cv = 5, n_jobs = -1)

grid_search.fit(x_train, y_train.ravel())
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
print("Best Accuracy: {:.2f} %".format(best_accuracy*100))
print("Best Parameters:", best_parameters)

In [ ]:
from sklearn.svm import SVR
regressor = SVR(kernel = 'rbf')
regressor.fit(x_train, y_train.ravel())

y_predicted_svr = regressor.predict(x_test)

# round predictions
y_predicted_svr = np.where(y_predicted < 0.5, 0, 1).reshape(-1,1)
y_predicted_svr

In [ ]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = regressor, X = x_train, y = y_train.ravel(), cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100)+"\n\n")

print(regressor.get_params().keys())

from sklearn.model_selection import GridSearchCV
parameters = [{'C': [1, 10, 100, 1000], 'gamma': [1e-3, 1e-4]}]
grid_search = GridSearchCV(estimator = regressor, param_grid = parameters, scoring = 'neg_root_mean_squared_error', cv = 5, n_jobs = -1)

grid_search.fit(x_train, y_train.ravel())
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
print("Best Accuracy: {:.2f} %".format(best_accuracy*100))
print("Best Parameters:", best_parameters)

In [ ]:
from sklearn.svm import SVR
regressor = SVR(kernel = 'rbf', C = 1, gamma = 0.0001)
regressor.fit(x_train, y_train.ravel())

y_predicted_svr = regressor.predict(x_test)

# round predictions
y_predicted_svr = np.where(y_predicted < 0.5, 0, 1).reshape(-1,1)
y_predicted_svr

In [ ]:
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor(random_state = 0)
regressor.fit(x_train, y_train)

y_predicted_dt = regressor.predict(x_test)

y_predicted_dt = np.where(y_predicted < 0.5, 0, 1).reshape(-1,1)
y_predicted_dt

In [ ]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = regressor, X = x_train, y = y_train.ravel(), cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100)+"\n\n")

print(regressor.get_params().keys())

from sklearn.model_selection import GridSearchCV
parameters = [{'max_depth':[3, None], 'max_features':[1, 2, 3, 4, 5], 'criterion': ['mse', 'mae']}]
grid_search = GridSearchCV(estimator = regressor, param_grid = parameters, scoring = 'neg_mean_squared_error', cv = 5, n_jobs = -1)

grid_search.fit(x_train, y_train.ravel())
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
print("Best Accuracy: {:.2f} %".format(best_accuracy*100))
print("Best Parameters:", best_parameters)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor(random_state = 0, max_depth = 3, max_features= 5)
regressor.fit(x_train, y_train)

y_predicted_dt = regressor.predict(x_test)

y_predicted_dt = np.where(y_predicted < 0.5, 0, 1).reshape(-1,1)
y_predicted_dt

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
regressor.fit(x_train, y_train.ravel())

y_predicted_rf = regressor.predict(x_test)

# round predictions
y_predicted_rf = np.where(y_predicted < 0.5, 0, 1).reshape(-1,1)
y_predicted_rf

In [ ]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = regressor, X = x_train, y = y_train.ravel(), cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100)+"\n\n")

from sklearn.model_selection import GridSearchCV
parameters = [{'n_estimators': [2, 4, 8, 16, 20], 'criterion': ['mse', 'mae']}]
grid_search = GridSearchCV(estimator = regressor,
param_grid = parameters, scoring = 'neg_root_mean_squared_error', cv = 5, n_jobs = -1)
grid_search.fit(x_train, y_train.ravel())
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
print("Best Accuracy: {:.2f} %".format(best_accuracy*100))
print("Best Parameters:", best_parameters)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 8, random_state = 0)
regressor.fit(x_train, y_train.ravel())
y_predicted_rf = regressor.predict(x_test)

# round predictions
y_predicted_rf = np.where(y_predicted < 0.5, 0, 1).reshape(-1,1)
# y_predicted = np.array([[round(x) for x in y_predicted]]).reshape(-1,1)
y_predicted_rf

In [ ]:
output = pd.DataFrame(df[df["Year"].str.contains("2020")])
output = output.drop(["Change_Net_Income", "Change_Cash_Operations", "Change_Cash_Investments", "Change_Cash_Financials", "Change_Cash_Equivalents", "Adj_Close"], axis = 1)
output = output.rename(columns = {"Year": "Date", "%Change":"y_true"})
output["y_true"] = np.where(output["y_true"] > 0.0, 1, 0)
output["y_predicted_dt"] = y_predicted_dt
output["y_predicted_rf"] = y_predicted_rf
output

In [ ]:
output.to_csv("regression.csv", index=False)